In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
%matplotlib inline

In [2]:
tf.__version__

'2.10.1'

In [3]:
df = pd.read_csv('./Hostel_Waste_Data.csv')

In [4]:
df.head()

,Prepared,Taken,Waste
0,950,710,240
1,1000,800,200
2,871,702,169
3,997,891,106
4,900,700,200


In [5]:
df.describe()

,Prepared,Taken,Waste
count,1782.000000,1782.000000,1782.000000
mean,911.125701,717.804714,193.320988
std,65.204674,67.829658,42.726376
min,780.000000,607.000000,50.000000
25%,850.000000,667.000000,180.000000
50%,900.000000,702.000000,200.000000
75%,980.000000,750.000000,200.000000
max,1000.000000,891.000000,370.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1782 entries, 0 to 1781
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Prepared  1782 non-null   int64
 1   Taken     1782 non-null   int64
 2   Waste     1782 non-null   int64
dtypes: int64(3)
memory usage: 41.9 KB


In [7]:
# Define a function to create the dataset for LSTM
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), :])
        y.append(data[i + time_steps, :])
    return np.array(X), np.array(y)

In [8]:
df['Waste'].shape

(1782,)

In [9]:
dataset = df['Waste'].values.reshape(1782,1)
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

In [10]:
# Determine the number of time steps
time_steps = 50  # Or you can set it based on your preference

# Create the dataset with appropriate time steps
X, y = create_dataset(dataset, time_steps)

In [11]:
len(X),len(y)

(1732, 1732)

In [12]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

In [13]:
X_train, X_test = X[0:train_size,:], X[train_size:len(X),:]
y_train, y_test = y[0:train_size,:], y[train_size:len(y),:]

In [14]:
X_train.shape, X_test.shape

((1385, 50, 1), (347, 50, 1))

In [15]:
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [16]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 50)                5050      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 45,901
Trainable params: 45,901
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
22/22 [==============================] - 5s 99ms/step - loss: 0.0807
Epoch 2/100
22/22 [==============================] - 2s 100ms/step - loss: 0.0188
Epoch 3/100
22/22 [==============================] - 2s 105ms/step - loss: 0.0183
Epoch 4/100
22/22 [==============================] - 2s 103ms/step - loss: 0.0183
Epoch 5/100
22/22 [==============================] - 2s 97ms/step - loss: 0.0179
Epoch 6/100
22/22 [==============================] - 2s 100ms/step - loss: 0.0179
Epoch 7/100
22/22 [==============================] - 2s 101ms/step - loss: 0.0181
Epoch 8/100
22/22 [==============================] - 2s 100ms/step - loss: 0.0181
Epoch 9/100
22/22 [==============================] - 2s 97ms/step - loss: 0.0181
Epoch 10/100
22/22 [==============================] - 2s 98ms/step - loss: 0.0182
Epoch 11/100
22/22 [==============================] - 2s 100ms/step - loss: 0.0179
Epoch 12/100
22/22 [==============================] - 2s 97ms/step - loss: 0.0179
Epoch 13/100
22/22

22/22 [==============================] - 2s 96ms/step - loss: 0.0175


In [19]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

11/11 [==============================] - 0s 15ms/step


In [20]:
mean_absolute_error(y_train,train_predict)

0.0809257721341474

In [21]:
mean_absolute_error(y_test,test_predict)

0.08652557548597155

In [22]:
# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform(y_train)
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform(y_test)

In [23]:
train_predict

array([[182.02728],
       [182.31697],
       [182.32089],
       ...,
       [183.37952],
       [183.47629],
       [183.44394]], dtype=float32)

In [24]:
y_train

array([[200.],
       [180.],
       [180.],
       ...,
       [220.],
       [200.],
       [200.]])

In [25]:
y_test

array([[200.],
       [200.],
       [200.],
       [170.],
       [200.],
       [200.],
       [180.],
       [260.],
       [370.],
       [131.],
       [220.],
       [260.],
       [106.],
       [170.],
       [200.],
       [150.],
       [180.],
       [189.],
       [180.],
       [180.],
       [200.],
       [330.],
       [278.],
       [180.],
       [200.],
       [200.],
       [131.],
       [180.],
       [180.],
       [180.],
       [140.],
       [180.],
       [200.],
       [370.],
       [170.],
       [200.],
       [185.],
       [200.],
       [200.],
       [200.],
       [200.],
       [260.],
       [170.],
       [180.],
       [155.],
       [166.],
       [200.],
       [200.],
       [200.],
       [200.],
       [170.],
       [180.],
       [153.],
       [180.],
       [200.],
       [180.],
       [200.],
       [180.],
       [170.],
       [180.],
       [140.],
       [163.],
       [200.],
       [200.],
       [200.],
       [169.],
       [18

In [26]:
test_predict

array([[183.78186],
       [184.29207],
       [184.88098],
       [185.48515],
       [186.20679],
       [187.4241 ],
       [187.98871],
       [188.2772 ],
       [188.73303],
       [192.2291 ],
       [194.96072],
       [193.11829],
       [191.59106],
       [194.73453],
       [194.06503],
       [192.7462 ],
       [190.68768],
       [190.1395 ],
       [189.22592],
       [188.01817],
       [187.07568],
       [186.38547],
       [189.20682],
       [188.13362],
       [184.186  ],
       [182.58395],
       [181.7749 ],
       [183.67166],
       [185.0998 ],
       [186.02536],
       [186.81908],
       [187.38309],
       [188.85043],
       [189.3086 ],
       [193.63766],
       [190.07431],
       [187.87888],
       [185.98067],
       [184.95663],
       [184.12051],
       [183.59006],
       [183.32678],
       [184.15408],
       [183.20007],
       [183.74707],
       [184.80594],
       [186.45642],
       [187.91592],
       [188.25203],
       [188.06552],


In [27]:
mean_absolute_error(y_train,train_predict)

25.896247472659777

In [28]:
mean_absolute_error(y_test,test_predict)

27.68818352339247

In [29]:
# Calculate root mean squared error
train_score = np.sqrt(mean_squared_error(y_train, train_predict))
test_score = np.sqrt(mean_squared_error(y_test, test_predict))
print('Train RMSE: %.2f' % (train_score))
print('Test RMSE: %.2f' % (test_score))

Train RMSE: 42.68
Test RMSE: 45.18


In [30]:
model.save('./model.h5')

In [31]:
model.save('./Models/Model1/')

INFO:tensorflow:Assets written to: ./Models/Model1/assets
